In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
from data_analysis import run_tsne, plot_tsne

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
data_source = 'C:/Users/BiSBII/Documents/MM_ML/data/'

In [ ]:
dataset_file = os.path.join(data_source, 'GREAT_TPM_GSE98923.csv')
metadata_file = os.path.join(data_source, 'GSE98923_metadata.xlsx')

In [ ]:
data = pd.read_csv(dataset_file, index_col=0)
metadata = pd.read_excel(metadata_file, index_col=0)

In [ ]:
data_log = pd.DataFrame(np.log2(data.values + 1.1), index=data.index, columns=data.columns)

In [ ]:
data_log.to_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923.csv'))

In [ ]:
data_model_file = 'C:/Users/BiSBII/Documents/plantdb/omics_data/RNAseq/UPDATE/GREAT_TPM_GSE98923_MODEL_GENES.csv'

In [ ]:
data_model = pd.read_csv(data_model_file, index_col=0)

In [ ]:
our_data_model = data_log.loc[data_model.index, :]
our_data_model.to_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923_MODEL_GENES.csv'))

# DATA ANALYSIS WILL ALL GENES

In [ ]:
data_log.shape

In [ ]:
data_log = data_log.transpose()
data_log.shape

In [ ]:
y_state = metadata['state'][data_log.index]
y_state

In [ ]:
values = y_state.value_counts()
print(values)

plt.pie(values , labels = values.index, autopct='%1.1f%%')
plt.savefig('pie_tissues.png')
plt.show()

In [ ]:
values = metadata['year'][data_log.index].value_counts()
print(values)

plt.pie(values , labels = values.index, autopct='%1.1f%%')
plt.savefig('pie_tissues.png')
plt.show()

In [ ]:
y_cv = metadata['cultivar'][data_log.index]

In [ ]:
values = y_cv.value_counts()
print(values)

plt.pie(values , labels = values.index, autopct='%1.1f%%')
plt.savefig('pie_tissues.png')
plt.show()

In [ ]:
# remove some features
vt = VarianceThreshold(0.1)
data_filtered = vt.fit_transform(data_log)
cols_inds = vt.get_support(indices=True)
df_data_filtered = pd.DataFrame(data_filtered, index=data_log.index, columns=data_log.columns[cols_inds])
df_data_filtered.shape

In [ ]:
df_tsne = run_tsne(n_components=2, data=df_data_filtered)
df_tsne

In [ ]:
df_tsne['factor'] = y_state

In [ ]:
plot_tsne(data=df_tsne, name_fig='tsne_all_genes_state', title='t-SNE projections of RNASeq data')

In [ ]:
df_tsne['factor'] = y_cv

In [ ]:
plot_tsne(data=df_tsne, name_fig='tsne_all_genes_cv', title='t-SNE projections of RNASeq data')

# Analyse data genes in model

In [ ]:
our_data_model = our_data_model.transpose()

In [ ]:
our_data_model.shape

In [ ]:
# remove some features
vt = VarianceThreshold(0.1)
data_filtered_model = vt.fit_transform(our_data_model)
cols_inds = vt.get_support(indices=True)
df_data_filtered_model = pd.DataFrame(data_filtered_model, index=our_data_model.index, columns=our_data_model.columns[cols_inds])
df_data_filtered_model.shape

In [ ]:
df_tsne_model = run_tsne(n_components=2, data=df_data_filtered_model)

In [ ]:
df_tsne_model['factor'] = y_state

In [ ]:
plot_tsne(data=df_tsne_model, name_fig='tsne_model_genes_state', title='t-SNE projections of RNASeq data')

In [ ]:
df_tsne_model['factor'] = y_cv

In [ ]:
plot_tsne(data=df_tsne_model, name_fig='tsne_model_genes_cv', title='t-SNE projections of RNASeq data')

# APPLY ML TO PREDICT LABEL STATE

Usar dataset sem replicates

In [ ]:
data_all_genes = pd.read_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923_NOREPS.csv'), index_col=0)
data_all_genes = data_all_genes.transpose()
data_all_genes.shape

Metadata without replicates

In [ ]:
metadata = pd.read_excel(metadata_file, index_col=0, sheet_name='NO_REPLICATES')
metadata

In [ ]:
y_state = metadata['state']
y_state

apply feature filter again

In [ ]:
# remove some features
vt = VarianceThreshold(0.1)
data_filtered = vt.fit_transform(data_all_genes)
cols_inds = vt.get_support(indices=True)
df_data_filtered = pd.DataFrame(data_filtered, index=data_all_genes.index, columns=data_all_genes.columns[cols_inds])
df_data_filtered.shape

Numero de features muito elevado, vamos selecionar 10000

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
kb = SelectKBest(f_classif, k=1000)

filt_kb = kb.fit_transform(df_data_filtered, y_state)

cols_inds = kb.get_support(indices=True)

df_kb = pd.DataFrame(filt_kb, columns=df_data_filtered.columns[cols_inds], index=df_data_filtered.index)
df_kb

divide data train test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_kb, y_state, test_size=0.20, random_state=42)

scale the data to facilitate the learning process

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Apply a simple model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, ConfusionMatrixDisplay

In [ ]:
print('Precision: %0.2f' % precision_score(y_test, y_pred, average='weighted'))
print('Recall: %0.2f' % recall_score(y_test, y_pred, average='weighted'))
print('Accuracy: %0.2f' % accuracy_score(y_test, y_pred))

In [ ]:
knn_cm = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=clf.classes_, cmap='Blues')
knn_cm

In [ ]:
from sklearn import svm

In [ ]:
svm_model = svm.SVC(kernel = "linear")
svm_model.fit(X_train, y_train)

svm_y_pred = svm_model.predict(X_test)

print('PECC (Accuracy): %0.2f' % svm_model.score(X_test, y_test))

print('Precision: %0.2f' % precision_score(y_test, svm_y_pred, average='weighted'))
print('Recall: %0.2f' % recall_score(y_test, svm_y_pred, average='weighted'))
print('Accuracy: %0.2f' % accuracy_score(y_test, svm_y_pred))

In [ ]:
svm_cm = ConfusionMatrixDisplay.from_predictions(y_test, svm_y_pred, display_labels=svm_model.classes_, cmap='Blues')
svm_cm

## Usar só os genes do modelo

load dataset model genes no reps

In [ ]:
data_model_genes = pd.read_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923_MODEL_GENES_NOREPS.csv'), index_col=0)
data_model_genes = data_model_genes.transpose()
data_model_genes.shape

divide train test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_model_genes, y_state, test_size=0.20, random_state=42)

In [ ]:
X_train.shape

apply filter

In [ ]:
# remove some features
vt = VarianceThreshold(0.1)
filter_train = vt.fit(X_train)

train_filtered = filter_train.transform(X_train)
test_filtered = filter_train.transform(X_test)

cols_inds = vt.get_support(indices=True)

X_train_filtered = pd.DataFrame(train_filtered, index=X_train.index, columns=X_train.columns[cols_inds])
X_train_filtered.shape

In [ ]:
X_test_filtered = pd.DataFrame(test_filtered, index=X_test.index, columns=X_test.columns[cols_inds])
X_test_filtered.shape

In [ ]:
kb2 = SelectKBest(f_classif, k=500)

kb2_fit = kb2.fit(X_train_filtered, y_train)

train_filtered2 = kb2_fit.transform(X_train_filtered)
test_filtered2 = kb2_fit.transform(X_test_filtered)

cols_inds = kb2_fit.get_support(indices=True)

X_train_filtered2 = pd.DataFrame(train_filtered2, columns=X_train_filtered.columns[cols_inds], index=X_train_filtered.index)
X_train_filtered2.shape

In [ ]:
X_test_filtered2 = pd.DataFrame(test_filtered2, columns=X_test_filtered.columns[cols_inds], index=X_test_filtered.index)
X_test_filtered2.shape

In [ ]:
scaler_model = StandardScaler().fit(X_train_filtered2)
X_train_scaled = scaler_model.transform(X_train_filtered2)
X_test_scaled = scaler_model.transform(X_test_filtered2)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered2.columns, index=X_train_filtered2.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered2.columns, index=X_test_filtered2.index)

In [ ]:
X_train_scaled_df.shape

In [ ]:
X_test_scaled_df.shape

In [ ]:
X_train_scaled_df.to_csv(os.path.join(data_source, 'XTRAIN_RNASEQ_MODEL_500_GENES_NOREPS.csv'))

In [ ]:
y_train.to_csv(os.path.join(data_source, 'yTRAIN_MODEL_500_GENES_NOREPS.csv'))

In [ ]:
X_test_scaled_df.to_csv(os.path.join(data_source, 'XTEST_RNASEQ_MODEL_500_GENES_NOREPS.csv'))

In [ ]:
y_test.to_csv(os.path.join(data_source, 'yTEST_MODEL_500_GENES_NOREPS.csv'))

# USAR ALL GENES (manter a divisão do split train / test anterior)

In [ ]:
data_all_genes = pd.read_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923_NOREPS.csv'), index_col=0)
data_all_genes = data_all_genes.transpose()
data_all_genes.shape

divide train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_all_genes, y_state, test_size=0.20, random_state=10)

In [ ]:
Xtrain_RNAseq_model = pd.read_csv(os.path.join('C:/Users/BiSBII/Documents/MM_ML/data', 'DIABLO_INPUT', 'XTRAIN_RNASEQ_MODEL_500_GENES_NOREPS.csv'), index_col=0)

In [ ]:
X_train = data_all_genes.loc[Xtrain_RNAseq_model.index, :]

In [ ]:
X_train.shape

In [ ]:
Xtest_RNAseq_model = pd.read_csv(os.path.join('C:/Users/BiSBII/Documents/MM_ML/data', 'DIABLO_INPUT', 'XTEST_RNASEQ_MODEL_500_GENES_NOREPS.csv'), index_col=0)

In [ ]:
X_test = data_all_genes.loc[Xtest_RNAseq_model.index, :]

In [ ]:
X_test.shape

In [ ]:
y_train_df = pd.read_csv(os.path.join('C:/Users/BiSBII/Documents/MM_ML/data', 'DIABLO_INPUT', 'yTRAIN_MODEL_500_GENES_NOREPS.csv'), index_col=0)
y_train = y_train_df['state']

In [ ]:
y_test_df = pd.read_csv(os.path.join('C:/Users/BiSBII/Documents/MM_ML/data', 'DIABLO_INPUT', 'yTEST_MODEL_500_GENES_NOREPS.csv'), index_col=0)
y_test = y_test_df['state']

In [ ]:
# remove some features
vt = VarianceThreshold(0.1)
filter_train = vt.fit(X_train)

train_filtered = filter_train.transform(X_train)
test_filtered = filter_train.transform(X_test)

cols_inds = vt.get_support(indices=True)

X_train_filtered = pd.DataFrame(train_filtered, index=X_train.index, columns=X_train.columns[cols_inds])
X_train_filtered.shape

In [ ]:
X_test_filtered = pd.DataFrame(test_filtered, index=X_test.index, columns=X_test.columns[cols_inds])
X_test_filtered.shape

In [ ]:
kb2 = SelectKBest(f_classif, k=500)

kb2_fit = kb2.fit(X_train_filtered, y_train)

train_filtered2 = kb2_fit.transform(X_train_filtered)
test_filtered2 = kb2_fit.transform(X_test_filtered)

cols_inds = kb2_fit.get_support(indices=True)

X_train_filtered2 = pd.DataFrame(train_filtered2, columns=X_train_filtered.columns[cols_inds], index=X_train_filtered.index)
X_train_filtered2.shape

In [ ]:
X_test_filtered2 = pd.DataFrame(test_filtered2, columns=X_test_filtered.columns[cols_inds], index=X_test_filtered.index)
X_test_filtered2.shape

In [ ]:
scaler_model = StandardScaler().fit(X_train_filtered2)
X_train_scaled = scaler_model.transform(X_train_filtered2)
X_test_scaled = scaler_model.transform(X_test_filtered2)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered2.columns, index=X_train_filtered2.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered2.columns, index=X_test_filtered2.index)

In [ ]:
X_train_scaled_df.shape

In [ ]:
X_test_scaled_df.shape

In [ ]:
X_train_scaled_df.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'XTRAIN_RNASEQ_ALL_500_GENES_NOREPS_NEWSPLIT.csv'))

In [ ]:
X_test_scaled_df.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'XTEST_RNASEQ_ALL_500_GENES_NOREPS_NEWSPLIT.csv'))

In [ ]:
y_train.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'yTRAIN_ALL_500_GENES_NOREPS_NEWSPLIT.csv'))

In [ ]:
y_test.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'yTEST_ALL_500_GENES_NOREPS_NEWSPLIT.csv'))

# fazer 5 splits

In [ ]:
skf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(skf.split(data_all_genes, y_state)):
    
    print(i)
    
    X_train = data_all_genes.iloc[train_index, :]
    X_test = data_all_genes.iloc[test_index, :]

    y_train = y_state.iloc[train_index]
    y_test = y_state.iloc[test_index]
    
    # remove some features
    vt = VarianceThreshold(0.1)
    filter_train = vt.fit(X_train)
    
    train_filtered = filter_train.transform(X_train)
    test_filtered = filter_train.transform(X_test)
    
    cols_inds = vt.get_support(indices=True)
    
    X_train_filtered = pd.DataFrame(train_filtered, index=X_train.index, columns=X_train.columns[cols_inds])
    X_test_filtered = pd.DataFrame(test_filtered, index=X_test.index, columns=X_test.columns[cols_inds])
    
    kb2 = SelectKBest(f_classif, k=500)

    kb2_fit = kb2.fit(X_train_filtered, y_train)

    train_filtered2 = kb2_fit.transform(X_train_filtered)
    test_filtered2 = kb2_fit.transform(X_test_filtered)

    cols_inds = kb2_fit.get_support(indices=True)

    X_train_filtered2 = pd.DataFrame(train_filtered2, columns=X_train_filtered.columns[cols_inds], index=X_train_filtered.index)
    X_test_filtered2 = pd.DataFrame(test_filtered2, columns=X_test_filtered.columns[cols_inds], index=X_test_filtered.index)
    
    scaler_model = StandardScaler().fit(X_train_filtered2)
    X_train_scaled = scaler_model.transform(X_train_filtered2)
    X_test_scaled = scaler_model.transform(X_test_filtered2)
    
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered2.columns, index=X_train_filtered2.index)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered2.columns, index=X_test_filtered2.index)
    
    X_train_scaled_df.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'XTRAIN_RNASEQ_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    X_test_scaled_df.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'XTEST_RNASEQ_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    y_train.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'yTRAIN_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    y_test.to_csv(os.path.join(data_source, 'DIABLO_INPUT', 'yTEST_ALL_500_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train_scaled_df, y_train)
y_pred = clf.predict(X_test_scaled_df)

In [ ]:
print('Precision: %0.2f' % precision_score(y_test, y_pred, average='weighted'))
print('Recall: %0.2f' % recall_score(y_test, y_pred, average='weighted'))
print('Accuracy: %0.2f' % accuracy_score(y_test, y_pred))

In [ ]:
cm_lr = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=clf.classes_, cmap='Blues')

In [ ]:
coefs = clf.coef_
feature_importance = pd.DataFrame(coefs.transpose(), index=X_train_scaled_df.columns, columns=['coef'])
feature_importance['mean_coef'] = abs(feature_importance).mean(axis=1)
feature_importance.sort_values(by=['mean_coef'], ascending=False)

In [ ]:
svm_model = svm.SVC(kernel = "linear")
svm_model.fit(X_train_scaled_df, y_train)

svm_y_pred = svm_model.predict(X_test_scaled_df)

print('PECC (Accuracy): %0.2f' % svm_model.score(X_test_scaled_df, y_test))

print('Precision: %0.2f' % precision_score(y_test, svm_y_pred, average='weighted'))
print('Recall: %0.2f' % recall_score(y_test, svm_y_pred, average='weighted'))
print('Accuracy: %0.2f' % accuracy_score(y_test, svm_y_pred))

In [ ]:
svm_cm = ConfusionMatrixDisplay.from_predictions(y_test, svm_y_pred, display_labels=svm_model.classes_, cmap='Blues')
svm_cm

In [ ]:
coefs = svm_model.coef_
feature_importance = pd.DataFrame(coefs.transpose(), index=X_train_scaled_df.columns, columns=['coef'])
feature_importance['mean_coef'] = abs(feature_importance).mean(axis=1)
feature_importance.sort_values(by=['mean_coef'], ascending=False)

In [ ]:
from sklearn.model_selection import cross_val_score, LeaveOneOut

In [ ]:
scores_svm_cv = cross_val_score(estimator=svm_model, X=data_filtered_model, y=y_state, cv = 5)
print('Accuracy values:', scores_svm_cv)
print('Mean accuracy: %0.2f' % scores_svm_cv.mean())

In [ ]:
loo_cv = LeaveOneOut()
scores_loo = cross_val_score(estimator=svm_model, X=data_filtered_model, y=y_state, cv=loo_cv)

print('Mean accuracy: %0.2f' % scores_loo.mean())

In [ ]:
coefs = svm_model.coef_
feature_importance = pd.DataFrame(coefs.transpose(), index=data_filtered_model.columns, columns=['coef'])
feature_importance['mean_coef'] = abs(feature_importance).mean(axis=1)
feature_importance.sort_values(by=['mean_coef'], ascending=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100)

# scores_rf = cross_val_score(rf_model, df_data_scaled_model, y_state, cv=5)

# print('Accuracy values:', scores_rf)
# print('Mean accuracy: %0.2f' % scores_rf.mean())

rf_model.fit(X_train_scaled_df, y_train)

rf_y_pred = rf_model.predict(X_test_scaled_df)

print('PECC (Accuracy): %0.2f' % rf_model.score(X_test_scaled_df, y_test))

print('Precision: %0.2f' % precision_score(y_test, rf_y_pred, average='weighted'))
print('Recall: %0.2f' % recall_score(y_test, rf_y_pred, average='weighted'))
print('Accuracy: %0.2f' % accuracy_score(y_test, rf_y_pred))

In [ ]:
importances = rf_model.feature_importances_
importances_df = pd.DataFrame(importances, columns=['importance'], index=X_train_scaled_df.columns)
importances_df.sort_values(by=['importance'], ascending=False)